In [10]:
import requests
from bs4 import BeautifulSoup
import time
import random

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
]


BASE_URL = "https://www3.animeflv.net/browse?page="  
ANIME_URL = "https://www3.animeflv.net"

page = 1
failures = 0 
links = []

while True:

    url = BASE_URL + str(page)
    headers = {'User-Agent': random.choice(user_agents)}

    try:

        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.content, 'lxml')
        anime_links = soup.find_all('article', class_='Anime alt B') 

        if len(anime_links) == 0:
                failures += 1
        else:
            failures = 0    
            for anime in anime_links:
                link = anime.find('a')['href']
                links.append(ANIME_URL + link)

        page += 1


    except Exception:
        failures += 1

    if failures >= 2:
        break

    time.sleep(random.randint(5, 10 ))

#remove duplicates
links = list(set(links))
    
with open('anime_links.txt', 'a') as f:
    for link in links:
        f.write(link + "\n")

print(f"Scraped {page - 1} pages")

Scraped 9 pages


In [1]:
import pandas as pd 
import re 
import requests
from bs4 import BeautifulSoup
import time
import random

In [7]:
#Only if you start from the list 

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
]


#Second part scrap the data from the links

data = []
check_point = 50 #checkpoint to save the data
links_processed = 0

# If resuming, load previously saved data and update links_processed

try:
    existing_data = pd.read_csv('anime_data.csv')
    data = existing_data.to_dict(orient='records')
    print(f'Resuming from link {len(data)}.')
except (FileNotFoundError, pd.errors.EmptyDataError):
    print('Starting from scratch.')

#Read the links from the file
with open('anime_links.txt', 'r') as f:
    links = f.read().splitlines()

# If we're resuming, skip the already processed links
if 'existing_data' in locals():
    links = links[len(data):]

for link in links: 

    try:

        url = link

        headers = {'User-Agent': random.choice(user_agents)}
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.content, 'lxml')

        # Code for year and episode number

        scripts = soup.find_all('script')

        # Initialize variables to store the information we are looking for
        last_episode_number = None
        last_episode_year = None

        # Now let's loop through all <script> elements and apply our regular expressions
        for script in scripts:
            if script.string:
                # Find the episodes array using a regular expression
                episodes_search = re.search(r'var episodes = \[\[(\d+),\d+\]', script.string)
                if episodes_search:
                    last_episode_number = episodes_search.group(1)

        #Varibles init here for titles 
        alt_title_1_text = None
        alt_title_2_text = None

        
        title = soup.find('h1', class_='Title').text
        span = soup.find('h1').find_next_sibling('span')
        tv_type = span.text

        alt_title_1 = soup.find('span', class_='TxtAlt')
        if alt_title_1:
            alt_title_1_text = alt_title_1.text.strip()
            alt_title_2 = alt_title_1.find_next_sibling('span', class_='TxtAlt')
            if alt_title_2:
                alt_title_2_text = alt_title_2.text.strip()
        else:
            alt_title_1 = None
            alt_title_2 = None
        rating = soup.find('span', class_='vtprmd')
        if rating:
            rating = rating.text
        else:
            rating = None
        description = soup.find('div', class_='Description').text
        description = description.replace('\n', '')
        genres = [a.text for a in soup.find('nav', class_='Nvgnrs').find_all('a')]
        followers = soup.find('div', class_='Top').find('div', class_='Title').text
        followers  = followers.replace('Seguidores: ', '')
        status = soup.find('span', class_='fa-tv').text
        

        data.append({
            'url': url,
            'title': title,
            'type_tv': tv_type,
            'alt_title_1': alt_title_1_text,
            'alt_title_2': alt_title_2_text,
            'rating': rating,
            'description': description,
            'genres': genres,
            'followers': followers,
            'status': status,
            'last_episode_number': last_episode_number
        })

    except Exception as e:
        print(f'Failed to process {link}: {type(e).__name__}, {e}')

    links_processed += 1

    if links_processed % check_point == 0:
        pd.DataFrame(data).to_csv('anime_data.csv', index=False)
        print(f'Saved checkpoint at link {links_processed}.')

    time.sleep(random.randint(1, 5))

# Save the remaining data if it's not an exact multiple of our checkpoint interval
if links_processed % check_point != 0:
    pd.DataFrame(data).to_csv('anime_data.csv', index=False)

print(f'Scraping complete. Processed {links_processed} links in total.')



Resuming from link 49.
Failed to process https://www3.animeflv.net/anime/kimi-no-iru-machi-uchi-ga-kita-machi: AttributeError, 'NoneType' object has no attribute 'text'
Failed to process https://www3.animeflv.net/anime/hokuto-no-ken-raoh-gaiden-junai-hen: AttributeError, 'NoneType' object has no attribute 'text'
Failed to process https://www3.animeflv.net/anime/yuyushiki: AttributeError, 'NoneType' object has no attribute 'text'
Saved checkpoint at link 50.
Saved checkpoint at link 100.
Failed to process https://www3.animeflv.net/anime/hikaru-no-go: AttributeError, 'NoneType' object has no attribute 'text'
Saved checkpoint at link 150.
Failed to process https://www3.animeflv.net/anime/special-a: AttributeError, 'NoneType' object has no attribute 'text'
Failed to process https://www3.animeflv.net/anime/mnemosyne: AttributeError, 'NoneType' object has no attribute 'text'
Saved checkpoint at link 200.
Failed to process https://www3.animeflv.net/anime/yami-shibai-6: AttributeError, 'NoneTy